<a href="https://colab.research.google.com/github/akibhaider/Kaggle_competition_guide/blob/main/BankersAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import sys
import numpy as np
from collections import defaultdict, deque
import pandas as pd
import itertools
import os

# from google.colab import drive
# drive.mount("/content/drive")
file_path = "/content/drive/MyDrive/Dataset_academics_learn/input0.csv"
# d = pd.read_csv(file_path, sep='\t')
# d = pd.read_csv(file_path, sep='\t', header=0)  # Automatically sets the first row as headers
# d = d.drop(0).reset_index(drop=True)

df = pd.read_csv(file_path, header=None)
d=df[0].str.split("\t",expand=True)
d.columns = d.iloc[0]
d = d.drop(0).reset_index(drop=True)

d.columns = d.columns.str.strip() ### Stripping spaces from column names

print("Columns in dataset:", d.columns.tolist()) ### testing column names

Columns in dataset: ['PID', 'Allocation_A', 'Allocation_B', 'Allocation_C', 'Max_A', 'Max_B', 'Max_C', 'Available_A', 'Available_B', 'Available_C', 'Need_A', 'Need_B', 'Need_C']


In [ ]:
### Column name check:
if 'PID' in d.columns:
    process = d['PID']
    PID = process.to_numpy()
else:
    print("Error: 'PID' column not found!")

process= d['PID']
PID = process.to_numpy()
allocation_columns = d.filter(regex="^Allocation_")
allocation_matrix = allocation_columns.astype(int).to_numpy()
Need_columns = d.filter(regex="^Need_")
Need_matrix = Need_columns.astype(int).to_numpy()
Max_columns = d.filter(regex="^Max_")
Max_matrix = Max_columns.astype(int).to_numpy()
Available_columns = d.filter(regex="^Available_")
row_to_extract = Available_columns.iloc[0]
Available_matrix = row_to_extract.astype(int).to_numpy()
x=0
if allocation_matrix.shape == Need_matrix.shape == Max_matrix.shape:
    if Need_matrix.shape[1] == Available_matrix.shape[0]:
      print("Valid File Format")
      x=1
    else:
      print("Invalid File Format")
else:
    print("Invalid File Format")

Valid File Format


In [ ]:
def find_safe_sequences(Need_matrix, Available_matrix, Allocation_matrix, processID, finish, work, safe_sequence, all_sequences):
    num_processes = len(processID)
    allocated = False
    for i in range(num_processes):
        if not finish[i] and all(Need_matrix[i] <= work):
            temp_work = work.copy()
            temp_work += Allocation_matrix[i]
            finish[i] = True
            safe_sequence.append(processID[i])
            find_safe_sequences(Need_matrix, temp_work, Allocation_matrix, processID, finish, temp_work, safe_sequence, all_sequences)
            finish[i] = False
            safe_sequence.pop()
            allocated = True
    if len(safe_sequence) == num_processes:
        all_sequences.append(safe_sequence.copy())

def bankers_algorithm_all_sequences(Need_matrix, Available_matrix, Allocation_matrix, processID):
    num_processes, num_resources = Need_matrix.shape
    finish = np.zeros(num_processes, dtype=bool)
    work = Available_matrix.copy()
    all_sequences = []
    print("\n🟢 Initial Available Resources:", work)
    find_safe_sequences(Need_matrix, work, Allocation_matrix, processID, finish, work, [], all_sequences)
    if all_sequences:
        print("\n✅ All possible safe sequences:")
        print("-------------------------------------")
        for i, sequence in enumerate(all_sequences):
          if i < 9:
              print("| " + f"{i + 1:02}:    {' => '.join(sequence)}" + " |")
          else:
              print("| " + f"{i + 1}:    {' => '.join(sequence)}" + " |")
          print("-------------------------------------")
    else:
        print("\n❌ No safe sequence found. Deadlock Occurred")
    return all_sequences



In [ ]:
if(x==1):
  all_safe_sequences = bankers_algorithm_all_sequences(Need_matrix, Available_matrix, allocation_matrix, PID)


🔵 Initial Available Resources: [3 3 2]

✅ All possible safe sequences:
-------------------------------------
| 01:    P1 => P3 => P0 => P2 => P4 |
-------------------------------------
| 02:    P1 => P3 => P0 => P4 => P2 |
-------------------------------------
| 03:    P1 => P3 => P2 => P0 => P4 |
-------------------------------------
| 04:    P1 => P3 => P2 => P4 => P0 |
-------------------------------------
| 05:    P1 => P3 => P4 => P0 => P2 |
-------------------------------------
| 06:    P1 => P3 => P4 => P2 => P0 |
-------------------------------------
| 07:    P1 => P4 => P3 => P0 => P2 |
-------------------------------------
| 08:    P1 => P4 => P3 => P2 => P0 |
-------------------------------------
| 09:    P3 => P1 => P0 => P2 => P4 |
-------------------------------------
| 10:    P3 => P1 => P0 => P4 => P2 |
-------------------------------------
| 11:    P3 => P1 => P2 => P0 => P4 |
-------------------------------------
| 12:    P3 => P1 => P2 => P4 => P0 |
----------------

In [ ]:
def bankers_algorithm(Need_matrix, Available_matrix, Allocation_matrix, Max_matrix, processID):
    num_processes, num_resources = Need_matrix.shape
    finish = np.zeros(num_processes, dtype=bool)
    safe_sequence = []
    work = Available_matrix.copy()
    print("\n🔵 Initial Available Resources:", work)
    while len(safe_sequence) < num_processes:
        allocated = False
        for i in range(num_processes):
            if not finish[i]:
                if all(Need_matrix[i] <= work):
                    print(f"\n✅ Process {processID[i]} is executing.")
                    print(f"🔄 Process {processID[i]} has finished. Releasing resources: {Allocation_matrix[i]}")
                    work += Allocation_matrix[i]
                    print(f"🟢 Updated Available Resources: {work}")
                    finish[i] = True
                    safe_sequence.append(processID[i])
                    allocated = True
                else:
                    print(f"❌ Process {processID[i]} cannot execute due to insufficient resources: {Need_matrix[i]} > {work}")
        if not allocated:
            print("\n⚠ No safe sequence found. Deadlock Occured")
            return None
    print("\n✅ Safe sequence found:", " -> ".join(safe_sequence))
    return safe_sequence

In [ ]:
if x==1:
  safe_sequence = bankers_algorithm(Need_matrix, Available_matrix, allocation_matrix, Max_matrix, PID)


🔵 Initial Available Resources: [3 3 2]
❌ Process P0 cannot execute due to insufficient resources: [7 4 3] > [3 3 2]

✅ Process P1 is executing.
🔄 Process P1 has finished. Releasing resources: [2 0 0]
🟢 Updated Available Resources: [5 3 2]
❌ Process P2 cannot execute due to insufficient resources: [6 0 0] > [5 3 2]

✅ Process P3 is executing.
🔄 Process P3 has finished. Releasing resources: [2 1 1]
🟢 Updated Available Resources: [7 4 3]

✅ Process P4 is executing.
🔄 Process P4 has finished. Releasing resources: [0 0 2]
🟢 Updated Available Resources: [7 4 5]

✅ Process P0 is executing.
🔄 Process P0 has finished. Releasing resources: [0 1 0]
🟢 Updated Available Resources: [7 5 5]

✅ Process P2 is executing.
🔄 Process P2 has finished. Releasing resources: [3 0 2]
🟢 Updated Available Resources: [10  5  7]

✅ Safe sequence found: P1 -> P3 -> P4 -> P0 -> P2
